# Pinecone Manager

## Uploading and Querying Documents

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.callbacks.manager import CallbackManager
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.embeddings import OllamaEmbeddings

from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
import pinecone
import random
import itertools

from credentials import OPENAI_API_KEY, PINECONE_API_KEY, PINECONE_API_ENV

/Users/nilsjennissen/CompanyProjects/streamlit_bot/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/nilsjennissen/CompanyProjects/streamlit_bot/venv/lib/python3.9/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [6]:
prompt = "What is the name of the school?"

In [7]:
# Load the PDF data
loader = PyPDFLoader("../docs/DM_Dir_NOF_CSM_14_set_2023.pdf")
docs = loader.load()
docs[:10]

[Document(page_content='NORMES\nD’ORGANITZACIÓ\nI\nFUNCIONAMENT\nDE\nCENTRE \n(NOFC)\nCOL·LEGI\nSANT\nMIQUEL\n[Aprovat\npel\nConsell\nEscolar\ndel\nCentre\nel\ndia\n14\nde\nsetembre\nde\n2023.]\n', metadata={'source': '../docs/DM_Dir_NOF_CSM_14_set_2023.pdf', 'page': 0}),
 Document(page_content='.\n2', metadata={'source': '../docs/DM_Dir_NOF_CSM_14_set_2023.pdf', 'page': 1}),
 Document(page_content="ÍNDEX\nI\nPART:\nREGLAMENT\nDE\nRÈGIM\nINTERN\n...........................................................................................\n6\nPREÀMBUL\n.......................................................................................................................................\n6\nTítol\npreliminar \nNATURALESA\nI\nFINALITAT\nDEL\nCOL·LEGI\n.........................................................................................\n7\nCapítol\n1r:\nDEFINICIÓ\nDEL\nCOL·LEGI\n.............................................................................................\n8\nCapítol\n2n

In [8]:
# Split the data into smaller documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=80)
chunks = text_splitter.split_documents(docs)
chunks[:10]

[Document(page_content='NORMES\nD’ORGANITZACIÓ\nI\nFUNCIONAMENT\nDE\nCENTRE \n(NOFC)\nCOL·LEGI\nSANT\nMIQUEL\n[Aprovat\npel\nConsell\nEscolar\ndel\nCentre\nel\ndia\n14\nde\nsetembre\nde\n2023.]', metadata={'source': '../docs/DM_Dir_NOF_CSM_14_set_2023.pdf', 'page': 0}),
 Document(page_content='.\n2', metadata={'source': '../docs/DM_Dir_NOF_CSM_14_set_2023.pdf', 'page': 1}),
 Document(page_content='ÍNDEX\nI\nPART:\nREGLAMENT\nDE\nRÈGIM\nINTERN\n...........................................................................................\n6\nPREÀMBUL\n.......................................................................................................................................\n6\nTítol\npreliminar \nNATURALESA\nI\nFINALITAT\nDEL\nCOL·LEGI\n.........................................................................................\n7\nCapítol\n1r:\nDEFINICIÓ\nDEL\nCOL·LEGI\n.............................................................................................\n8\nCapítol\n2n:\

## Embeddings and Upload

In [9]:
# Create embeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Create a Pinecone index and add the documents to it
#pinecone.create_index("python-index", dimension=1536, metric="cosine")
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)
index_name = "python-index"
docsearch = Pinecone.from_texts([t.page_content for t in chunks], embeddings, index_name=index_name)

## Docsearch with Pinecone

In [10]:
# Perform similarity search
docs = docsearch.similarity_search(prompt)

# Load the question answering chain
llm = Ollama(model="mistral", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))
chain = load_qa_chain(llm, chain_type="stuff")

# Query the documents and get the answer
answer = chain.run(input_documents=docs, question=prompt)
answer


The name of the school is "El Col·legi Sant Miquel"

'\nThe name of the school is "El Col·legi Sant Miquel"'

## Connecting to Pinecone directly

In [11]:
# Listing the existing indexes
pinecone.list_indexes()

['python-index']

In [12]:
# Connecting to an existing index
index = pinecone.Index("python-index")

In [13]:
# Get information about the index
pinecone.describe_index("python-index")

IndexDescription(name='python-index', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='p1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

## Upserting documents

In [39]:
# Test docs
test_docs = docs[:2]
test_docs

[Document(page_content='  \n Col·legi Sant Miquel  \nUna institució centenària que mira al futur  \n \n \n \nEL COR DEL NOSTRE PROJECTE EDUCATIU  \n \nCOL·LEGI SANT MIQUEL \n \n \nUna institució centenària que mira al futur  \n \nMés de cent anys d’història  \n \nL’any 1898 els Missioners del Sagrat Cor van crear el  Col·legi Sant \nMiquel amb la finalitat de realitzar la seva missió e vangelitzadora \nmitjançant l’acció educativa al servei dels habitants de Barcelona i els seus voltants.  \nDes de llavors i de manera ininterrompuda, milers d’alumnes han passat per les nostres aules i és a ells justament que hem tractat de \nservir , tot oferint -los una educació de qualitat que, més enllà de la \nsimple instrucció acadèmica, els ha inculcat  uns valors  humans i \ncristians  que els han ajudat  a ser, a més de bons professionals, \nhomes i dones per als altres, ciutadans honrats  i transmissors dels \nvalors evangèlics.  \nL’any 1998 vam celebrar el Centenari i vam aprofitar per recor

In [44]:
# Extract chunks
chunks = [t.page_content for t in test_docs]
chunks

['  \n Col·legi Sant Miquel  \nUna institució centenària que mira al futur  \n \n \n \nEL COR DEL NOSTRE PROJECTE EDUCATIU  \n \nCOL·LEGI SANT MIQUEL \n \n \nUna institució centenària que mira al futur  \n \nMés de cent anys d’història  \n \nL’any 1898 els Missioners del Sagrat Cor van crear el  Col·legi Sant \nMiquel amb la finalitat de realitzar la seva missió e vangelitzadora \nmitjançant l’acció educativa al servei dels habitants de Barcelona i els seus voltants.  \nDes de llavors i de manera ininterrompuda, milers d’alumnes han passat per les nostres aules i és a ells justament que hem tractat de \nservir , tot oferint -los una educació de qualitat que, més enllà de la \nsimple instrucció acadèmica, els ha inculcat  uns valors  humans i \ncristians  que els han ajudat  a ser, a més de bons professionals, \nhomes i dones per als altres, ciutadans honrats  i transmissors dels \nvalors evangèlics.  \nL’any 1998 vam celebrar el Centenari i vam aprofitar per recordar les nostres senyes